In [1]:
# Load necessary packages and the dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Load dataset
df = pd.read_csv(r'house_price.csv')
df.head()

,Id,Neighborhood,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,GrLivArea,FullBath,...,Heating,Electrical,Foundation,Exterior1st,Exterior2nd,RoofStyle,SaleCondition,MoSold,YrSold,SalePrice
0,1,Richmond,Duplex,4.0,5.0,1938,2007,1418.542110,1334.672273,2.0,...,Electric,Mix,BrkTil,Wood,Wood,Cedar,Normal,4.0,2024.0,165523.670390
1,2,Delta,Duplex,7.0,1.0,1919,-9223372036854775808,570.622787,844.691015,1.0,...,HeatPump,FuseF,BrkTil,Vinyl,Vinyl,Cedar,Abnorml,NaN,2021.0,165344.625514
2,3,Burnaby,BasementSuite,4.0,6.0,1981,1970,816.080798,2030.685080,1.0,...,Radiant,FuseA,CBlock,Stucco,Stucco,Asphalt,Abnorml,6.0,2022.0,262609.986627
3,4,Surrey,BasementSuite,5.0,5.0,1932,-9223372036854775808,589.704590,625.410833,2.0,...,Radiant,SBrkr,PConc,Vinyl,Vinyl,TorchOn,Partial,4.0,2021.0,134246.920554
4,5,Surrey,Detached,6.0,NaN,1957,1994,1052.067733,1427.795371,3.0,...,Gas,FuseA,Stone,Stucco,Stucco,Asphalt,Normal,9.0,2020.0,167186.521789


In [2]:
df = df.ffill()

In [3]:
# Remove outliers using IQR method
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1

# Create masks for lower and upper bounds, applying only to numeric columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter only numeric columns to avoid alignment issues
numeric_columns = df.select_dtypes(include=['number']).columns
mask = ~((df[numeric_columns] < lower_bound) | (df[numeric_columns] > upper_bound)).any(axis=1)

# Apply the mask to the original dataframe
df_clean = df[mask]
df_clean.reset_index(drop=True, inplace=True)
df_clean.describe()

,Id,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,...,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,LotFrontage,LotArea,MasVnrArea,MoSold,YrSold,SalePrice
count,2130.000000,2130.000000,2130.000000,2130.000000,2130.00000,2130.000000,2130.000000,2130.000000,2130.0,2129.000000,...,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000,2130.000000
mean,2508.384507,5.950235,4.961972,1969.604225,1989.79108,1052.314281,1483.950674,1.995305,1.0,2.996712,...,5.904695,1.089202,1.961502,493.117808,70.695587,10046.518273,110.822811,5.992019,2020.976526,181638.614996
std,1461.476349,1.534437,1.487685,29.481979,24.20701,338.795672,477.336525,1.032573,0.0,0.985081,...,1.884357,0.889649,1.031865,199.090706,23.790041,2867.275097,85.081592,2.804317,1.534147,48427.665221
min,1.000000,2.000000,1.000000,1887.000000,1921.00000,114.338365,400.000000,0.000000,1.0,1.000000,...,2.000000,0.000000,0.000000,0.000000,20.000000,2086.397810,0.000000,1.000000,2017.000000,48403.452665
25%,1210.250000,5.000000,4.000000,1948.000000,1973.00000,809.638555,1156.028381,1.000000,1.0,2.000000,...,5.000000,0.000000,1.000000,354.331270,53.324244,8070.839675,35.715898,4.000000,2020.000000,148694.962679
50%,2529.500000,6.000000,5.000000,1970.000000,1990.00000,1062.676026,1488.888716,2.000000,1.0,3.000000,...,6.000000,1.000000,2.000000,491.148432,69.981067,10101.079780,104.549096,6.000000,2021.000000,182014.067677
75%,3786.750000,7.000000,6.000000,1991.000000,2006.00000,1286.444133,1809.571293,3.000000,1.0,4.000000,...,7.000000,2.000000,3.000000,633.370422,87.118281,12039.373913,171.827963,8.000000,2022.000000,215321.790365
max,4996.000000,10.000000,9.000000,2052.000000,2056.00000,1979.854922,2821.167255,4.000000,1.0,5.000000,...,10.000000,3.000000,4.000000,1032.767709,136.684541,17957.572944,374.056412,12.000000,2024.000000,315785.745471
